# Natural Language Processing 
 #### 12/20/2019

In [237]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.preprocessing.text as tfkpt
from tensorflow.keras.preprocessing.text import Tokenizer
import csv
import json



In [238]:
hotels_df = pd.read_csv('7282_1.csv')

hotels_df.head(5)

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [239]:
rank_hotels_df = hotels_df[['reviews.rating', 'reviews.text']].sort_values(by = 'reviews.rating')
rank_hotels_df.describe()

,reviews.rating
count,35050.000000
mean,3.776431
std,1.416195
min,0.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,10.000000


In [240]:
rank_hotels_df.shape

(35912, 2)

In [241]:
# Check examples of what rating 0.0 means
rank_hotels_df[rank_hotels_df['reviews.rating']== 0.0]

,reviews.rating,reviews.text
35911,0.0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
29873,0.0,to share your opinion of this businesswith YP ...
29874,0.0,to share your opinion of this businesswith YP ...
20524,0.0,to share your opinion of this businesswith YP ...
10802,0.0,to share your opinion of this businesswith YP ...
...,...,...
13409,0.0,to share your opinion of this businesswith YP ...
21713,0.0,to share your opinion of this businesswith YP ...
97,0.0,to share your opinion of this businesswith YP ...
35651,0.0,to share your opinion of this businesswith YP ...


In [242]:
unique_ratings = rank_hotels_df['reviews.rating'].unique()
unique_ratings

array([ 0. ,  1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  3.1,
        3.2,  3.3,  3.4,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.4,  5.8,  6. ,  6.3,  6.7,
        7. ,  7.1,  7.5,  7.9,  8.3,  8.8,  9.2,  9.5,  9.6, 10. ,  nan])

### To Clean Data:
1. remove all data with rating 0 from dataset
2. remove all data without rating from dataset
3. dataset has two rating system in it. one is from 1 to 5, another is 1.0 to 10.0. There is less data with system from 1.0 to 10.0 so I will need to cut some of the data to make data more sutable for sentiment-classifier model.
4. Everything that has rating more that 9.0 will be considered as positeve(2). Everything that has rating 5 - positive. Everything between 5 and 9.0 wil be removed from dataset
5. Everything that has rating less than 3.0 will be considered as negative (0). Everything that has rating 3 or less - negative. All floats number from 3.0 to 9.0 will be removed
6. All reviews wth rating "4" will be considered as neutral (1)




#### Remove all data with rating 0 from dataset

In [243]:
hotels_df_without0 = rank_hotels_df[rank_hotels_df['reviews.rating']!= 0.0]
hotels_df_without0['reviews.rating'].unique()

array([ 1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9,  5. ,  5.4,  5.8,  6. ,  6.3,  6.7,  7. ,
        7.1,  7.5,  7.9,  8.3,  8.8,  9.2,  9.5,  9.6, 10. ,  nan])

#### Remove all data without rating from dataset

In [244]:
hotels_df_without_nan = hotels_df_without0.dropna()
hotels_df_without_nan['reviews.rating'].unique()

array([ 1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9,  5. ,  5.4,  5.8,  6. ,  6.3,  6.7,  7. ,
        7.1,  7.5,  7.9,  8.3,  8.8,  9.2,  9.5,  9.6, 10. ])

#### Remove data with certain ratings that won't work for our model

In [245]:
arr_ratings_exclude = [ 3.1,  3.2, 3.3,  3.4,  3.7,  3.8,  3.9, 4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9, 5.4,  5.8,  6. ,  6.3,  6.7,  7., 7.1,  7.5,  7.9,  8.3,  8.8]

hotels_df_clear = hotels_df_without_nan
for el in arr_ratings_exclude:
    hotels_df_clear = hotels_df_clear[hotels_df_clear['reviews.rating']!= el]
    
hotels_df_clear['reviews.rating'].unique()
    

array([ 1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  4. ,  5. ,
        9.2,  9.5,  9.6, 10. ])

In [246]:
hotels_df_clear.shape

(34374, 2)

##### Original dataset had 35912 rows so after data cleaning we have 34374 rows which means that we lost 1538 rows of data or 4.28% of original data.

In [247]:
def create_sentiment(num):
    """function to assign 0(negative), 1(neutral), 2(positive) based on ratings number"""
    if num <= 3:
        return 0
    if num == 4:
        return 1
    if num >= 5:
        return 2

In [248]:
hotels_df_clear['sentiment'] = hotels_df_clear['reviews.rating'].apply(create_sentiment)
hotels_df_clear.sample(20)

,reviews.rating,reviews.text,sentiment
34779,4.0,Not bad except for the lack of wifi in the roo...,1
29715,3.0,the breakfast was okay but the person taking c...,0
26661,5.0,For visitors to Venice who are budget consciou...,2
33884,4.0,"It is handy to an o.k. restaraunt, staff was q...",1
27603,1.0,I pulled in at 1 am with two small children ju...,0
5328,2.0,"Prior to staying at this hotel, I stayed in th...",0
15274,3.0,"Ok, in need of upgrades.",0
20160,5.0,We had to travel to Glasgow for a family weddi...,2
29261,5.0,I was staying at the Marriott Marquis this wee...,2
20721,4.0,We really enjoyed the seafood buffet and a won...,1


In [249]:
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 0


In [250]:
# Positive sentiments amount
hotels_df_clear[hotels_df_clear['sentiment'] == 2].shape

(12868, 3)

In [251]:
# Neutral sentiments amount
hotels_df_clear[hotels_df_clear['sentiment'] == 1].shape

(9747, 3)

In [252]:
# Negative sentiments amount
hotels_df_clear[hotels_df_clear['sentiment'] == 0].shape

(11759, 3)

In [253]:
hotels_df_x=hotels_df_clear['reviews.text'].tolist()
hotels_df_y= hotels_df_clear['sentiment'].tolist()


In [254]:
tokenizer  = Tokenizer()

In [255]:
tokenizer.fit_on_texts(hotels_df_x)


In [256]:
hotels_df_x = tokenizer.texts_to_sequences(hotels_df_x)


In [257]:
print(type(hotels_df_x))

<class 'list'>


In [258]:
hotels_df_x = tokenizer.sequences_to_matrix(hotels_df_x)
hotels_df_x

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [259]:
print(type(hotels_df_x))

<class 'numpy.ndarray'>


In [260]:
model = keras.Sequential()
# model.add(Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(677, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(677, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(3, activation = 'softmax'))

In [261]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [262]:
hotels_df_y=keras.utils.to_categorical(hotels_df_y, 3)
hotels_df_y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [264]:
model.fit(hotels_df_x, hotels_df_y,
          batch_size=3500, 
          epochs=7,
          verbose=1,
          validation_split=0.2, 
          shuffle=True)


Train on 27499 samples, validate on 6875 samples
Epoch 1/7
27499/27499 [==============================] - 60s 2ms/sample - loss: 0.9820 - accuracy: 0.4997 - val_loss: 1.3670 - val_accuracy: 0.0000e+00
Epoch 2/7
27499/27499 [==============================] - 52s 2ms/sample - loss: 0.7637 - accuracy: 0.6491 - val_loss: 1.2768 - val_accuracy: 0.4423
Epoch 3/7
27499/27499 [==============================] - 49s 2ms/sample - loss: 0.6163 - accuracy: 0.7435 - val_loss: 1.2603 - val_accuracy: 0.4556
Epoch 4/7
27499/27499 [==============================] - 50s 2ms/sample - loss: 0.4850 - accuracy: 0.8063 - val_loss: 1.5621 - val_accuracy: 0.3908
Epoch 5/7
27499/27499 [==============================] - 50s 2ms/sample - loss: 0.3740 - accuracy: 0.8561 - val_loss: 1.6980 - val_accuracy: 0.4131
Epoch 6/7
27499/27499 [==============================] - 73s 3ms/sample - loss: 0.2707 - accuracy: 0.9009 - val_loss: 1.7929 - val_accuracy: 0.4566
Epoch 7/7
27499/27499 [==============================] - 79

In [266]:
reviews_model = model.to_json()
with open("hotels_reviews_model.json", "w") as json_file:
    json_file.write(reviews_model)
model.save_weights("hotels_reviews_model.h5")